In [0]:
"""
A CDC - Change Data Capture / Replication on Going project.
A MySQL database will be placed in AWS RDS. 
AWS DMS will take data from RDS and write that data into temporary S3 bucket.
Next Lambda funtion will be triggered for every new object stored in temp S3 bucket 
and invoke PySpark Job in AWS Glue.
All the changes to the RDS DB will be eventually replicated to the final S3 storage.
"""

In [0]:
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CDC").getOrCreate()

In [0]:
# final load data frame
fldf = spark.read.csv("/FileStore/tables/LOAD00000001.csv")
fldf = fldf.withColumnRenamed("_c0", "id").withColumnRenamed("_c1", "FullName").withColumnRenamed("_c2", "City")
fldf.write.mode("overwrite").csv("/FileStore/tables/finalOutput/finalFile.csv")

In [0]:
# updated data frame
udf = spark.read.csv("/FileStore/tables/20220625_164447274.csv")
udf = udf.withColumnRenamed("_c0","action").withColumnRenamed("_c1", "id").withColumnRenamed("_c2", "FullName").withColumnRenamed("_c3", "City")
# final file data frame
ffdf = spark.read.csv("/FileStore/tables/finalOutput/finalFile.csv")
ffdf = ffdf.withColumnRenamed("_c0", "id").withColumnRenamed("_c1", "FullName").withColumnRenamed("_c2", "City")

In [0]:
for row in udf.collect(): 
    if row["action"] == 'U':
        ffdf = ffdf.withColumn("FullName", when(ffdf["id"] == row["id"], row["FullName"]).otherwise(ffdf["FullName"]))      
        ffdf = ffdf.withColumn("City", when(ffdf["id"] == row["id"], row["City"]).otherwise(ffdf["City"]))
    
    if row["action"] == 'I':
        insertedRow = [list(row)[1:]]
        columns = ['id', 'FullName', 'City']
        newdf = spark.createDataFrame(insertedRow, columns)
        ffdf = ffdf.union(newdf)
    
    if row["action"] == 'D':
        ffdf = ffdf.filter(ffdf.id != row["id"])
        
    
        
